In [1]:
import urllib
import pytz
import pandas_datareader.data as web
import datetime
from bs4 import BeautifulSoup
import csv

In [2]:
thisurl = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies" # the wikipedia url containing list of S&P 500 companie
myPage = urllib.request.urlopen(thisurl) # opens this url

mySoup = BeautifulSoup(myPage, "html.parser") # parse html soup 

table = mySoup.find('table', {'class': 'wikitable sortable'}) # finds wiki sortable table in webpage html

sector_tickers = dict() # create a dictionary to store all tickers according to sector
for row in table.findAll('tr'): # find every row in the table
    col = row.findAll('td') # find every column in that row
    if len(col) > 0 : # if there are columns in that row
        sector = str(col[3].string.strip()).lower().replace(' ', '_') # identify the sector in the row
        if col[0] is not None:
          ticker = col[0] # identify the ticker in the row
        if sector not in sector_tickers: # if this sector is not a key in the dictionary
            sector_tickers[sector] = list() # add this as a key to the dictionary
        sector_tickers[sector].append(ticker) # add the ticker to right key in the dictionary
# for i in sector_tickers:
  # print (i,sector_tickers[i])

In [3]:
#### Section 2: Queries Yahoo Finance for historical data on tickers

# Start and end dates for historical data
start = datetime.datetime(2015, 1, 1)  # start date
end = datetime.datetime(2020, 12, 31) # end date

myKeys = sector_tickers.keys() # find all the sectors which are keys in the dictionary we created in Step 1
myKeys = (list(myKeys))
for i in range(0,len(myKeys)): # for each key in the dictionary which represents a sector
    # print (myKeys[i])
    myTickers = sector_tickers[myKeys[i]] # find the tickers in that list
    for j in range(0,len(myTickers)): # for each ticker
        # print (myTickers[j].a.text)
        try:
          myData = web.DataReader(myTickers[j].a.text, 'yahoo', start, end) # query the pandas datareader to pull data from Yahoo! finance
          fileName = myTickers[j].a.text + '.csv' # create a file
          myData.to_csv("/content/sample_data/files/"+fileName) # save data to the file
        except Exception as e:
          pass

In [4]:
import os

print ((os.listdir("/content/sample_data/files")))

['ACN.csv', 'CHRW.csv', 'MRK.csv', 'ABC.csv', 'SPGI.csv', 'NWS.csv', 'IR.csv', 'CMS.csv', 'AVB.csv', 'SJM.csv', 'NFLX.csv', 'HFC.csv', 'SO.csv', 'REG.csv', 'C.csv', 'AMAT.csv', 'YUM.csv', 'FCX.csv', 'IT.csv', 'BIO.csv', 'IEX.csv', 'DD.csv', 'WY.csv', 'MTB.csv', 'AVY.csv', 'GOOG.csv', 'TJX.csv', 'DOW.csv', 'AXP.csv', 'IVZ.csv', 'DG.csv', 'AVGO.csv', 'WFC.csv', 'PBCT.csv', 'AAL.csv', 'BEN.csv', 'IDXX.csv', 'GM.csv', 'XRAY.csv', 'PSX.csv', 'NVR.csv', 'D.csv', 'HBI.csv', 'AME.csv', 'DXCM.csv', 'VFC.csv', 'BSX.csv', 'QCOM.csv', 'F.csv', 'ODFL.csv', 'HWM.csv', 'WMT.csv', 'MU.csv', 'O.csv', 'LMT.csv', 'UDR.csv', 'MLM.csv', 'ICE.csv', 'CL.csv', 'ED.csv', 'MGM.csv', 'KEY.csv', 'VNO.csv', 'UNH.csv', 'AES.csv', 'SLG.csv', 'AAPL.csv', 'TDG.csv', 'ANSS.csv', 'GPS.csv', 'PHM.csv', 'KMX.csv', 'J.csv', 'BKNG.csv', 'DVA.csv', 'PLD.csv', 'PPG.csv', 'MA.csv', 'MRO.csv', 'GE.csv', 'UA.csv', 'DRI.csv', 'MKTX.csv', 'MS.csv', 'CFG.csv', 'GILD.csv', 'DPZ.csv', 'WMB.csv', 'BAX.csv', 'DLTR.csv', 'LUMN.csv', 'US

In [5]:
import shutil
shutil.make_archive("index", 'zip', "/content/sample_data/files")

'/content/index.zip'

In [6]:
import pandas as pd
import numpy as np

files=os.listdir("/content/sample_data/files")
data=[]
for j in files:
  x=pd.read_csv("/content/sample_data/files/"+j)
  li=[0.00]
  closed=list(x.Close)
  for i in range(1,len(closed)):
    per=((closed[i-1]-closed[i])/closed[i-1])*100
    li.append(per)
  x['%Gain or %Loss']=li
  avg=sum(li)/len(li)
  data.append([j.split(".")[0],avg])
  x.to_csv("/content/sample_data/Daywise/"+j)
  df = pd.DataFrame(data, columns = ['StockName', 'AvgGain'])
  df.to_csv("/content/sample_data/avggain.csv")


In [7]:
avggain=pd.read_csv("/content/sample_data/avggain.csv")
avggain.sort_values(by=['AvgGain'], inplace=True)
avggain.to_csv("/content/sample_data/avggain.csv")
avggain

,Unnamed: 0,StockName,AvgGain
182,182,CARR,-0.652483
468,468,AMD,-0.312127
224,224,VNT,-0.306874
380,380,NVDA,-0.256543
463,463,TSLA,-0.244170
...,...,...,...
434,434,NOV,0.055484
285,285,PRGO,0.056257
194,194,FTI,0.058269
469,469,SLB,0.058787


In [8]:
least25=avggain[:25]
least25.to_csv("/content/sample_data/bottom25.csv")

In [9]:
top25=avggain[len(avggain)-25:]
top25[::-1].to_csv("/content/sample_data/top25.csv")